In [183]:
import pandas as pd
import numpy as np
import json

In [184]:
df = pd.read_excel('Data/3de3_CDMX/Secretaria_de_Gobierno/03_Norma_Angelica_Licona_Alberto/01_DP_NALA.xlsx',index_col=0, header=None)
df.reset_index(inplace=True, drop=True)
print(df)

                                                        1   \
0                                                      NaN   
1           Declaración patrimonial de la iniciativa #3de3   
2                                                      NaN   
3        Favor de NO eliminar ninguna fila o celda del ...   
4                                                      NaN   
5                                  I. DATOS DEL DECLARANTE   
6        En esta sección se encuentra la información qu...   
7                                                  Nombre:   
8                                                   Poder:   
9                                                  Ámbito:   
10                      Entidad de gobierno o institución:   
11                           Cargo que desempeña o aspira:   
12                                                     NaN   
13       II. DATOS DEL CÓNYUGE, CONCUBINA/CONCUBINARIO ...   
14       Complete la siguiente información que identifi...   
15      

In [179]:
def all_null(row):
    if len(row.unique())==1:
        if np.isnan(row.unique()[0]):
            return 'drop'
    return 'keep'

In [180]:
df['act'] = df.apply(lambda row: all_null(row), axis=1)
df = df[df['act']=='keep']
df.reset_index(inplace=True, drop=True)

KeyboardInterrupt: 

In [146]:
tables_num = ['I.','II.','III.','IV.','V.', 'VI.']
def sections(x):
    if (type(x)!=float)&(type(x)!=int):
        for i in tables_num:
            #print x, i
            if x.split(' ')[0]==i:
                return i
    return np.nan
df['section'] = df[1].apply(lambda x: sections(x))   

In [147]:
x='drop'
x_l=[]
for i in df['section'].tolist():
    if type(i)==str:
        x=i
    x_l.append(x)
df['section']=x_l

In [148]:
def check_empty(d):
    l_t = []
    if d.shape[0]==0:
        for i in range(d.shape[1]):
            l_t.append(np.nan)
        d.loc[0] = l_t
    return d

In [149]:
df.shape

(87, 12)

### I Section

In [150]:
sec_1 = df[df['section']=='I.']
sec_1.reset_index(inplace=True, drop=True)

In [151]:
sec_1_1 = sec_1.iloc[::,:2:].T
sec_1_1.drop([0,1], axis=1, inplace=True)
sec_1_1.columns = sec_1_1.loc[1]
sec_1_1.drop(index=1, inplace=True)
sec_1_1.reset_index(inplace=True, drop=True)

In [152]:
sec_1_2 =sec_1.iloc[2::,5:7:].T
sec_1_2.columns = sec_1_2.loc[6]
sec_1_2.drop(index=6, inplace=True)
sec_1_2.reset_index(inplace=True, drop=True)
sec_1_final = pd.concat([sec_1_1,sec_1_2], axis=1)
sec_1_final.head()

,Nombre:,Poder:,Ámbito:,Entidad de gobierno o institución:,Cargo que desempeña o aspira:,Estado civil:,Entidad federativa:,Municipio o Delegación:,Distrito electoral:,Partido político
0,José Antonio Peña Merino,Ejecutivo,Estatal,Dirección General de Gobernabilidad de Tecnolo...,Titular,Casado/Casada,Distrito Federal,No aplica,NaN,NaN


### II Section

In [153]:
sec_2 = df[df['section']=='II.'].copy(deep=True)
sec_2.reset_index(inplace=True, drop=True)
sec_2.shape

(7, 12)

In [154]:
sec_2.columns = sec_2.loc[2][:6].tolist() + sec_2.loc[3][6:].tolist()
sec_2 = sec_2.iloc[5::,:-2:]
col2=[u'Enliste e indique la relaci\xf3n entre el declarante con el c\xf3nyuge, concubina o concubinario y todo dependiente econ\xf3mico',
 u'Nacionalidad',
 u'\xbfHa desempe\xf1ado un cargo de gobierno \nen los \xfaltimos 5 a\xf1os?',
 u'Dependencia o entidad p\xfablica',
 u'Cargo, puesto o funci\xf3n',
 u'Periodo']
sec_2 = sec_2[col2]
sec_2 = check_empty(sec_2)
sec_2.reset_index(inplace=True, drop=True)
sec_2.head()

,"Enliste e indique la relación entre el declarante con el cónyuge, concubina o concubinario y todo dependiente económico",Nacionalidad,¿Ha desempeñado un cargo de gobierno en los últimos 5 años?,Dependencia o entidad pública,"Cargo, puesto o función",Periodo
0,Cónyuge,MÉXICO,No,NaN,NaN,NaN
1,Padre/Madre,MÉXICO,No,NaN,NaN,NaN


### III Section

In [155]:
sec_3 = df[df['section']=='III.'].copy(deep=True)
sec_3.reset_index(inplace=True, drop=True)
sec_3.shape

(21, 12)

In [156]:
subsets = ['1.','1.1']
def sub_sections(x):
    if (type(x)!=float)&(type(x)!=int):
        for i in subsets:
            #print x, i
            if x.split(' ')[0]==i:
                return i
    return np.nan
sec_3['section'] = sec_3[1].apply(lambda x: sub_sections(x))

In [157]:
x='drop'
x_l=[]
for i in sec_3['section'].tolist():
    if type(i)==str:
        x=i
    x_l.append(x)
sec_3['section']=x_l
sec_3 = sec_3[sec_3['section']!='drop']

In [158]:
sec_3_1 = sec_3[sec_3['section']=='1.']
sec_3_1.reset_index(inplace=True, drop=True)
sec_3_1 = sec_3_1.iloc[2::,::8].T.copy(deep=True)
sec_3_1.columns = sec_3_1.loc[1]
sec_3_1.drop(1, axis=0, inplace=True)
col_3_1=[u'Remuneraci\xf3n neta anual del declarante por cargos p\xfablicos:',
# u'Nota 1: Incluye sueldos, honorarios, compensaciones, bonos y otras prestaciones.',
# u'Otros ingresos del declarante:',
 u'\u2013 Por actividades industrial, empresarial o comercial',
# u'Nota 2: Se refiere a ingresos por actividades industriales, empresariales o comerciales en M\xe9xico y en el extranjero.',
 u'\u2013 Por actividad financiera',
# u'Nota 3: Se refiere a ingresos por rendimiento de valores o contratos bancarios, plusval\xedas de participaci\xf3n accionaria e ingresos por pr\xe9stamos.           ',
 u'\u2013 Por servicios profesionales',
# u'Nota 4: Los ingresos por servicios profesionales pueden incluir la participaci\xf3n en consejos, consultor\xedas o asesor\xedas de forma permanente u ocasional, en M\xe9xico y en el extranjero.',
 u'\u2013 Por otras actividades',
# u'Nota 5: Se refiere a ingresos por arrendamientos, regal\xedas, sorteos, concursos, donaciones, entre otros.',
 u'Ingreso neto anual total del declarante:']
sec_3_1 = sec_3_1[col_3_1]
sec_3_1 = check_empty(sec_3_1)
sec_3_1.reset_index(inplace=True, drop=True)
sec_3_1.head()

1,Remuneración neta anual del declarante por cargos públicos:,"– Por actividades industrial, empresarial o comercial",– Por actividad financiera,– Por servicios profesionales,– Por otras actividades,Ingreso neto anual total del declarante:
0,0,1170000,0,198000,0,1368000


In [159]:
sec_3_1_1 = sec_3[sec_3['section']=='1.1']
sec_3_1_1.reset_index(inplace=True, drop=True)
sec_3_1_1 = sec_3_1_1.iloc[::,:-3:8].T.copy(deep=True)
sec_3_1_1.columns = sec_3_1_1.loc[1]
col3_1_1=[#u'1.1 Ingreso anual neto del del C\xd3NYUGE, CONCUBINA O CONCUBINARIO Y/O DEPENDIENTES ECON\xd3MICOS entre el 1\xb0 de enero y el 31 de diciembre del a\xf1o inmediato anterior',
 #u'El ingreso neto anual del c\xf3nyuge, concubina o concubinario y dependientes econ\xf3micos se refiere a los ingresos netos \u2013despu\xe9s de impuestos\u2013 que tuvieron en el \xfaltimo a\xf1o fiscal concluido por actividades en cargos p\xfablicos, actividad industrial o comercial, actividad financiera, servicios profesionales \u2013incluyendo participaciones en consejos, consultor\xedas o asesor\xedas\u2013 as\xed como cualquier otro ingreso por actividades diversas. No es necesario desagregar por tipo de ingreso. Las cifras expresadas en esta secci\xf3n se declara en moneda nacional (MXN).',
 u'Ingreso anual neto del c\xf3nyuge, concubina o concubinario:',
 u'Ingreso anual neto de otros dependientes econ\xf3micos:',
 u'Ingreso neto anual total del c\xf3nyuge, concubina o concubinario y/o dependientes econ\xf3micos:']
sec_3_1_1 = sec_3_1_1[col3_1_1]
sec_3_1_1.drop(1, axis=0, inplace=True)
sec_3_1_1 = check_empty(sec_3_1_1)
sec_3_1_1.reset_index(inplace=True, drop=True)
sec_3_1_1.head()

1,"Ingreso anual neto del cónyuge, concubina o concubinario:",Ingreso anual neto de otros dependientes económicos:,"Ingreso neto anual total del cónyuge, concubina o concubinario y/o dependientes económicos:"
0,240000,14000,254000


### IV Section

In [160]:
sec_4 = df[df['section']=='IV.'].copy(deep=True)
sec_4.reset_index(inplace=True, drop=True)
sec_4.shape

(40, 12)

In [176]:
subsets = ['1.', '1.1','2.', '2.1','3.', '3.1','4.','5.']
sec_4['section'] = sec_4[1].apply(lambda x: sub_sections(x))
print(sec_4['section'])

0      1.
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     1.1
8     NaN
9     NaN
10    NaN
11    NaN
12     2.
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    2.1
19    NaN
20    NaN
21     3.
22    NaN
23    NaN
24    NaN
25    NaN
26    3.1
27    NaN
28    NaN
29    NaN
30    NaN
31     4.
32    NaN
33    NaN
34    NaN
35    NaN
36    NaN
37    NaN
Name: section, dtype: object


In [162]:
x='drop'
x_l=[]
for i in sec_4['section'].tolist():
    if type(i)==str:
        x=i
    x_l.append(x)
sec_4['section']=x_l
sec_4 = sec_4[sec_4['section']!='drop']
sec_4.reset_index(inplace=True, drop=True)

In [163]:
sec_4_1 = sec_4[sec_4['section']=='1.']
sec_4_1.reset_index(inplace=True, drop=True)
sec_4_1.columns = sec_4_1.loc[2].tolist()[:1] + sec_4_1.loc[3].tolist()[1:3] + sec_4_1.loc[2].tolist()[3:]

In [164]:
sec_4_1 = sec_4_1.iloc[4:-1:,:-2:]
sec_4_1 = check_empty(sec_4_1)
sec_4_1.reset_index(inplace=True, drop=True)
sec_4_1.head()

,Tipo de bien declarado,Terreno,Construcción,País donde se encuentra el bien inmueble,Entidad federativa y municipio o delegación donde se encuentra el bien inmueble,Forma de operación en la que se adquirió el bien inmueble,Año en que se realizó la adquisición del bien inmueble,Valor del bien inmueble conforme a escritura pública,Moneda,Titular del bien inmueble declarado
0,Terreno,4000,0,MÉXICO,Tulancingo; Hidalgo,Contado,2016,300000,MXN – Nuevos pesos,Declarante
1,Terreno,5000,0,MÉXICO,"Tenango de Doria, Hidalgo",Contado,2017,400000,MXN – Nuevos pesos,Declarante


In [165]:
sec_4_1_1 = sec_4[sec_4['section']=='1.1']
sec_4_1_1.reset_index(inplace=True, drop=True)
sec_4_1_1.columns = sec_4_1_1.loc[2]
sec_4_1_1 = sec_4_1_1.iloc[4:-1:,:-5:]

col4_1_1=[u'Tipo de bien declarado',
 u'Pa\xeds donde se encuentra el bien inmueble',
 u'Entidad federativa y municipio o delegaci\xf3n donde se encuentra el bien inmueble',
 u'Forma de operaci\xf3n en la que se adquiri\xf3 el bien inmueble',
 u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del bien inmueble',
 u'Titular del bien inmueble declarado']
sec_4_1_1 = sec_4_1_1[col4_1_1]
sec_4_1_1 = check_empty(sec_4_1_1)
sec_4_1_1.reset_index(inplace=True, drop=True)
sec_4_1_1.head()

2,Tipo de bien declarado,País donde se encuentra el bien inmueble,Entidad federativa y municipio o delegación donde se encuentra el bien inmueble,Forma de operación en la que se adquirió el bien inmueble,Año en que se realizó la adquisición del bien inmueble,Titular del bien inmueble declarado
0,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
sec_4_2 = sec_4[sec_4['section']=='2.']
sec_4_2.reset_index(inplace=True, drop=True)
sec_4_2.columns = sec_4_2.loc[2]
sec_4_2 = sec_4_2.iloc[4:-1:,:-2:]
col4_2=[u'Tipo de veh\xedculo',
 u'Marca',
 u'Modelo',
 u'Pa\xeds donde est\xe1 registrado el veh\xedculo',
 u'Forma de operaci\xf3n en la que se adquiri\xf3 el veh\xedculo',
 u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del veh\xedculo',
 u'Valor del veh\xedculo',
 u'Moneda',
 u'Titular del veh\xedculo']
sec_4_2 = sec_4_2[col4_2]
sec_4_2 = check_empty(sec_4_2)
sec_4_2.reset_index(inplace=True, drop=True)
sec_4_2.head()

2,Tipo de vehículo,Marca,Modelo,País donde está registrado el vehículo,Forma de operación en la que se adquirió el vehículo,Año en que se realizó la adquisición del vehículo,Valor del vehículo,Moneda,Titular del vehículo
0,Automotor,Ford,EcoSport,MÉXICO,Crédito,2015,263000,MXN – Nuevos pesos,Declarante


In [167]:
sec_4_2_1 = sec_4[sec_4['section']=='2.1']
sec_4_2_1.reset_index(inplace=True, drop=True)
sec_4_2_1.columns = sec_4_2_1.loc[2]
sec_4_2_1 = sec_4_2_1.iloc[3::,:-6:]
col4_2_1=[u'Tipo de veh\xedculo',
 u'Pa\xeds donde est\xe1 registrado el veh\xedculo',
 u'Forma de operaci\xf3n en la que se adquiri\xf3 el veh\xedculo',
 u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del veh\xedculo',
 u'Titular del veh\xedculo']
sec_4_2_1 = sec_4_2_1[col4_2_1]
sec_4_2_1 = check_empty(sec_4_2_1)
sec_4_2_1.reset_index(inplace=True, drop=True)
sec_4_2_1.head()

2,Tipo de vehículo,País donde está registrado el vehículo,Forma de operación en la que se adquirió el vehículo,Año en que se realizó la adquisición del vehículo,Titular del vehículo
0,NaN,NaN,NaN,NaN,NaN


In [168]:
sec_4_3 = sec_4[sec_4['section']=='3.']
sec_4_3.reset_index(inplace=True, drop=True)
sec_4_3.columns = sec_4_3.loc[2]
sec_4_3 = sec_4_3.iloc[4:-1:,:-4:]
col4_3=[u'Tipo de bien mueble',
 u'Forma de operaci\xf3n en la que se adquiri\xf3 el bien mueble',
 u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del bien mueble',
 u'Valor estimado del bien mueble',
 u'Moneda',
 u'Titular del bien mueble']
sec_4_3 = sec_4_3[col4_3]
sec_4_3 = check_empty(sec_4_3)
sec_4_3.reset_index(inplace=True, drop=True)
sec_4_3.head()

2,Tipo de bien mueble,Forma de operación en la que se adquirió el bien mueble,Año en que se realizó la adquisición del bien mueble,Valor estimado del bien mueble,Moneda,Titular del bien mueble
0,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
sec_4_3_1 = sec_4[sec_4['section']=='3.1']
sec_4_3_1.reset_index(inplace=True, drop=True)
sec_4_3_1.columns = sec_4_3_1.loc[2]
sec_4_3_1 = sec_4_3_1.iloc[4:-1:,:-7:]
col4_3_1=[u'Tipo de bien mueble',
 u'Forma de operaci\xf3n en la que se adquiri\xf3 el bien mueble',
 u'A\xf1o en que se realiz\xf3 la adquisici\xf3n del bien mueble',
 u'Titular del bien mueble']
sec_4_3_1 = sec_4_3_1[col4_3_1]
sec_4_3_1 = check_empty(sec_4_3_1)
sec_4_3_1.reset_index(inplace=True, drop=True)
sec_4_3_1.head()

2,Tipo de bien mueble,Forma de operación en la que se adquirió el bien mueble,Año en que se realizó la adquisición del bien mueble,Titular del bien mueble
0,NaN,NaN,NaN,NaN


In [170]:
sec_4_4 = sec_4[sec_4['section']=='4.']
sec_4_4.reset_index(inplace=True, drop=True)
sec_4_4.columns = sec_4_4.loc[2].tolist()[:5] + sec_4_4.loc[3].tolist()[5:8] + sec_4_4.loc[2].tolist()[8:]
sec_4_4 = sec_4_4.iloc[4:-1:,:-2:]
col4_4=[u'Tipo de inversi\xf3n',
 u'Pa\xeds donde est\xe1 constituida la entidad donde se encuentra la inversi\xf3n',
 u'Nombre o raz\xf3n social de la entidad',
 u'Menor o igual a $100,000.00',
 u'Entre $100,000.01 y $500,000.00',
 u'Mayor o igual a $500,000.01',
 u'Moneda de la inversi\xf3n',
 u'Titularidad de la inversi\xf3n']
sec_4_4 = sec_4_4[col4_4]
sec_4_4 = check_empty(sec_4_4)
sec_4_4.reset_index(inplace=True, drop=True)
sec_4_4.head()

,Tipo de inversión,País donde está constituida la entidad donde se encuentra la inversión,Nombre o razón social de la entidad,"Menor o igual a $100,000.00","Entre $100,000.01 y $500,000.00","Mayor o igual a $500,000.01",Moneda de la inversión,Titularidad de la inversión
0,Bancaria,MÉXICO,Banorte,"Menor o igual a $100,000.00",NaN,NaN,MXN – Nuevos pesos,Declarante
1,Bancaria,MÉXICO,Banorte,"Menor o igual a $100,000.00",NaN,NaN,MXN – Nuevos pesos,Declarante


### V Section

In [171]:
sec_5 = df[df['section']=='V.']
sec_5.reset_index(inplace=True, drop=True)
sec_5.columns = sec_5.loc[2]
sec_5 = sec_5.loc[3::,::].copy(deep=True)
sec_5.reset_index(inplace=True, drop=True)
sec_5 = sec_5.iloc[:-2:,:-6:]

In [172]:
def all_5_nan(row):
    row = row[1:]
    if len(row.unique())==1:
        return 'drop'
    else:
        return 'keep'
sec_5['tag'] = sec_5.apply(lambda row: all_5_nan(row), axis=1)
sec_5 = sec_5[sec_5['tag'] =='keep'].copy(deep=True)
sec_5.drop('tag', axis=1,inplace=True)
col4_5=[u'No. de nota',
 u'Secci\xf3n a la que hace referencia',
 u'Subsecci\xf3n a la que hace referencia',
 u'Descripci\xf3n de la nota aclaratoria']
sec_5 = sec_5[col4_5]
sec_5 = check_empty(sec_5)
sec_5.reset_index(inplace=True, drop=True)
sec_5.head()

2,No. de nota,Sección a la que hace referencia,Subsección a la que hace referencia,Descripción de la nota aclaratoria
0,1,IV. DATOS SOBRE BIENES,"IV - 1.1 Bienes inmuebles del CÓNYUGE, CONCUBI...",Ambos terrenos se encuentran en proceso de cam...


In [173]:
table_list = {'sec_1':sec_1_final, 
'sec_2':sec_2, #'sec_2_1': sec_2_1_1,'sec_2_2':sec_2_2, 'sec_2_2_1':sec_2_2_1,'sec_2_3':sec_2_3,'sec_2_3_1':sec_2_3_1,'sec_2_4':sec_2_4,'sec_2_5':sec_2_5, 
'sec_3_1':sec_3_1, 'sec_3_1_1':sec_3_1_1,# 'sec_3_3':sec_3_3, 
'sec_4_1':sec_4_1, 'sec_4_1_1':sec_4_1_1, 'sec_4_2':sec_4_2, 'sec_4_2_1':sec_4_2_1, 'sec_4_3':sec_4_3, 'sec_4_3_1':sec_4_3_1,  'sec_4_4':sec_4_4, #'sec_4_5':sec_4_5,
'sec_5':sec_5}

In [174]:
for key in table_list.keys():
    print(key)
    table_list[key].to_csv('Output/Patrimonio/'+key+'pat.csv', index=0,encoding='utf-8' )

sec_1
sec_2
sec_3_1
sec_3_1_1
sec_4_1
sec_4_1_1
sec_4_2
sec_4_2_1
sec_4_3
sec_4_3_1
sec_4_4
sec_5


In [175]:
the_json = {sec_1_final['Nombre:'].values[0] + '_' + str(df[10][0]).split(' ')[0]:
 {'sec_1':sec_1_final.to_json(),
  'sec_2':sec_2.to_json(),
  'sec_3':{
    'sec_3_1':sec_3_1.to_json(), 'sec_3_1_1':sec_3_1_1.to_json(),
  },
  'sec_4':{
      'sec_4_1':sec_4_1.to_json(), 'sec_4_1_1':sec_4_1_1.to_json(), 'sec_4_2':sec_4_2.to_json(), 
      'sec_4_2_1':sec_4_2_1.to_json(), 'sec_4_3':sec_4_3.to_json(), 'sec_4_3_1':sec_4_3_1.to_json(),  'sec_4_4':sec_4_4.to_json()
  },
  'sec_5':sec_5.to_json(),
 }  
}

In [140]:
with open('Output/patrimonio_'+sec_1_final['Nombre:'].values[0].replace(' ','') + '_' + str(df[10][0]).split(' ')[0] + '.json', 'w') as fp:
    json.dump(the_json, fp)